In [1]:
import gym
import numpy as np

from gym.wrappers import AtariPreprocessing
gym.new_step_api=True
env = gym.make('Pong-v0')

H = 200 # number of hidden layer neurons
D = 80 * 80 # input dimensionality: 80x80 grid
model = {}
model['W1'] = np.random.randn(H,D) / np.sqrt(D) # "Xavier" initialization
model['W2'] = np.random.randn(H) / np.sqrt(H)
# hyperparameters
batch_size = 10 # every how many episodes to do a param update?
learning_rate = 1e-3
gamma = 0.99 # discount factor for reward
decay_rate = 0.99 # decay factor for RMSProp leaky sum of grad^2
grad_buffer = { k : np.zeros_like(v) for k,v in model.items() } # update buffers that add up gradients over a batch
rmsprop_cache = { k : np.zeros_like(v) for k,v in model.items() } # rmsprop memory

def sigmoid(x): 
  return 1.0 / (1.0 + np.exp(-x)) # sigmoid "squashing" function to interval [0,1]

def prepro(I):
  I=np.asarray(I)
  I = I[35:195] # crop
  I = I[::2,::2,0] # downsample by factor of 2
  
  I[I == 144] = 0 # erase background (background type 1)
  
  I[I == 109] = 0 # erase background (background type 2)
  
  I[I != 0] = 1 # everything else (paddles, ball) just set to 1
  return I.astype(float).ravel()

def discount_rewards(r):
  discounted_r = np.zeros_like(r)
  running_add = 0
  for t in reversed(range(0, r.size)):
    if r[t] != 0: running_add = 0 # reset the sum, since this was a game boundary (pong specific!)
    running_add = running_add * gamma + r[t]
    discounted_r[t] = running_add
  return discounted_r

def policy_forward(x):
  h = np.dot(model['W1'], x)
  h[h<0] = 0 # ReLU nonlinearity
  logp = np.dot(model['W2'], h)
  p = sigmoid(logp)
  return p, h # return probability of taking action 2, and hidden state

def policy_backward(epx, eph, epdlogp):
  """ backward pass. (eph is array of intermediate hidden states) """
  dW2 = np.dot(eph.T, epdlogp).ravel()
  dh = np.outer(epdlogp, model['W2'])
  dh[eph <= 0] = 0 # backpro prelu
  dW1 = np.dot(dh.T, epx)
  return {'W1':dW1, 'W2':dW2}

def model_step(model, observation, prev_x):

  cur_x = prepro(observation)
  x = cur_x - prev_x if prev_x is not None else np.zeros(D)
  prev_x = cur_x
  
  # forward the policy network and sample an action from the returned probability
  aprob, _ = policy_forward(x)
  action = 2 if aprob >= 0.5 else 3 # roll the dice!
  
  return action, prev_x

def play_game(env, model):
  observation = env.reset()

  frames = []
  cumulated_reward = 0

  prev_x = None # used in computing the difference frame

  for t in range(1000):
      frames.append(env.render(mode = 'rgb_array'))
      action, prev_x = model_step(model, observation, prev_x)
      observation, reward, done, info = env.step(action)
      cumulated_reward += reward
      if done:
          print("Episode finished after {} timesteps, accumulated reward = {}".format(t+1, cumulated_reward))
          break
  print("Episode finished without success, accumulated reward = {}".format(cumulated_reward))
  env.close()
  display_frames_as_gif(frames)

def train_model(env, model, total_episodes = 100):
  hist = []
  observation = env.reset()

  prev_x = None # used in computing the difference frame
  xs,hs,dlogps,drs = [],[],[],[]
  running_reward = None
  reward_sum = 0
  episode_number = 0

  while True:
  
    cur_x = prepro(observation)
    x = cur_x - prev_x if prev_x is not None else np.zeros(D)
    prev_x = cur_x

    # forward the policy network and sample an action from the returned probability
    aprob, h = policy_forward(x)
    action = 2 if np.random.uniform() < aprob else 3 # roll the dice!

    # record various intermediates (needed later for backprop)
    xs.append(x) # observation
    hs.append(h) # hidden state
    y = 1 if action == 2 else 0 # a "fake label"
    dlogps.append(y - aprob) # grad that encourages the action that was taken to be taken (see http://cs231n.github.io/neural-networks-2/#losses if confused)

    # step the environment and get new measurements
    observation, reward, done, info = env.step(action)
    reward_sum += reward

    drs.append(reward) # record reward (has to be done after we call step() to get reward for previous action)

    if done: # an episode finished
      episode_number += 1

      # stack together all inputs, hidden states, action gradients, and rewards for this episode
      epx = np.vstack(xs)
      eph = np.vstack(hs)
      epdlogp = np.vstack(dlogps)
      epr = np.vstack(drs)
      xs,hs,dlogps,drs = [],[],[],[] # reset array memory

      # compute the discounted reward backwards through time
      discounted_epr = discount_rewards(epr)
      # standardize the rewards to be unit normal (helps control the gradient estimator variance)
      discounted_epr -= np.mean(discounted_epr)
      discounted_epr /= np.std(discounted_epr)

      epdlogp *= discounted_epr # modulate the gradient with advantage (PG magic happens right here.)
      grad = policy_backward(epx, eph, epdlogp)
      for k in model: grad_buffer[k] += grad[k] # accumulate grad over batch

      # perform rmsprop parameter update every batch_size episodes
      if episode_number % batch_size == 0:
        for k,v in model.items():
          g = grad_buffer[k] # gradient
          rmsprop_cache[k] = decay_rate * rmsprop_cache[k] + (1 - decay_rate) * g**2
          model[k] += learning_rate * g / (np.sqrt(rmsprop_cache[k]) + 1e-5)
          grad_buffer[k] = np.zeros_like(v) # reset batch gradient buffer

      # boring book-keeping
      running_reward = reward_sum if running_reward is None else running_reward * 0.99 + reward_sum * 0.01
      hist.append((episode_number, reward_sum, running_reward))
      print ('episode %f, reward total was %f. running mean: %f' % (episode_number, reward_sum, running_reward))
      reward_sum = 0
      observation = env.reset() # reset env
      prev_x = None
      if episode_number == total_episodes: 
        return hist

   
    

C:\Users\manas\AppData\Local\Programs\Python\Python310\lib\site-packages\gym\envs\registration.py:592: UserWarning: WARN: The environment Pong-v0 is out of date. You should consider upgrading to version `v4`.
  logger.warn(
C:\Users\manas\AppData\Local\Programs\Python\Python310\lib\site-packages\gym\core.py:329: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
C:\Users\manas\AppData\Local\Programs\Python\Python310\lib\site-packages\gym\wrappers\step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


In [2]:
%time hist1 = train_model(env, model, total_episodes=7000)

C:\Users\manas\AppData\Local\Programs\Python\Python310\lib\site-packages\gym\utils\passive_env_checker.py:227: DeprecationWarning: WARN: Core environment is written in old step API which returns one bool instead of two. It is recommended to rewrite the environment with new step API. 
  logger.deprecation(


episode 1.000000, reward total was -21.000000. running mean: -21.000000
episode 2.000000, reward total was -21.000000. running mean: -21.000000
episode 3.000000, reward total was -20.000000. running mean: -20.990000
episode 4.000000, reward total was -21.000000. running mean: -20.990100
episode 5.000000, reward total was -21.000000. running mean: -20.990199
episode 6.000000, reward total was -21.000000. running mean: -20.990297
episode 7.000000, reward total was -20.000000. running mean: -20.980394
episode 8.000000, reward total was -21.000000. running mean: -20.980590
episode 9.000000, reward total was -21.000000. running mean: -20.980784
episode 10.000000, reward total was -20.000000. running mean: -20.970976
episode 11.000000, reward total was -20.000000. running mean: -20.961267
episode 12.000000, reward total was -20.000000. running mean: -20.951654
episode 13.000000, reward total was -21.000000. running mean: -20.952137
episode 14.000000, reward total was -21.000000. running mean

episode 114.000000, reward total was -20.000000. running mean: -20.526205
episode 115.000000, reward total was -19.000000. running mean: -20.510943
episode 116.000000, reward total was -20.000000. running mean: -20.505833
episode 117.000000, reward total was -19.000000. running mean: -20.490775
episode 118.000000, reward total was -21.000000. running mean: -20.495867
episode 119.000000, reward total was -21.000000. running mean: -20.500909
episode 120.000000, reward total was -21.000000. running mean: -20.505900
episode 121.000000, reward total was -20.000000. running mean: -20.500841
episode 122.000000, reward total was -21.000000. running mean: -20.505832
episode 123.000000, reward total was -21.000000. running mean: -20.510774
episode 124.000000, reward total was -20.000000. running mean: -20.505666
episode 125.000000, reward total was -21.000000. running mean: -20.510609
episode 126.000000, reward total was -21.000000. running mean: -20.515503
episode 127.000000, reward total was -

episode 225.000000, reward total was -21.000000. running mean: -20.266297
episode 226.000000, reward total was -20.000000. running mean: -20.263634
episode 227.000000, reward total was -21.000000. running mean: -20.270997
episode 228.000000, reward total was -21.000000. running mean: -20.278287
episode 229.000000, reward total was -21.000000. running mean: -20.285504
episode 230.000000, reward total was -21.000000. running mean: -20.292649
episode 231.000000, reward total was -19.000000. running mean: -20.279723
episode 232.000000, reward total was -21.000000. running mean: -20.286926
episode 233.000000, reward total was -20.000000. running mean: -20.284056
episode 234.000000, reward total was -19.000000. running mean: -20.271216
episode 235.000000, reward total was -21.000000. running mean: -20.278504
episode 236.000000, reward total was -21.000000. running mean: -20.285719
episode 237.000000, reward total was -21.000000. running mean: -20.292861
episode 238.000000, reward total was -

episode 336.000000, reward total was -19.000000. running mean: -20.053859
episode 337.000000, reward total was -21.000000. running mean: -20.063320
episode 338.000000, reward total was -21.000000. running mean: -20.072687
episode 339.000000, reward total was -19.000000. running mean: -20.061960
episode 340.000000, reward total was -21.000000. running mean: -20.071340
episode 341.000000, reward total was -18.000000. running mean: -20.050627
episode 342.000000, reward total was -20.000000. running mean: -20.050121
episode 343.000000, reward total was -16.000000. running mean: -20.009620
episode 344.000000, reward total was -21.000000. running mean: -20.019523
episode 345.000000, reward total was -21.000000. running mean: -20.029328
episode 346.000000, reward total was -20.000000. running mean: -20.029035
episode 347.000000, reward total was -21.000000. running mean: -20.038745
episode 348.000000, reward total was -19.000000. running mean: -20.028357
episode 349.000000, reward total was -

...

episode 6512.000000, reward total was 7.000000. running mean: 0.649053
episode 6513.000000, reward total was -3.000000. running mean: 0.612562
episode 6514.000000, reward total was -4.000000. running mean: 0.566437
episode 6515.000000, reward total was 9.000000. running mean: 0.650772
episode 6516.000000, reward total was 12.000000. running mean: 0.764265
episode 6517.000000, reward total was 2.000000. running mean: 0.776622
episode 6518.000000, reward total was 13.000000. running mean: 0.898856
episode 6519.000000, reward total was -2.000000. running mean: 0.869867
episode 6520.000000, reward total was -5.000000. running mean: 0.811169
episode 6521.000000, reward total was 8.000000. running mean: 0.883057
episode 6522.000000, reward total was 10.000000. running mean: 0.974226
episode 6523.000000, reward total was -3.000000. running mean: 0.934484
episode 6524.000000, reward total was 4.000000. running mean: 0.965139
episode 6525.000000, reward total was 11.000000. running mean: 1.0654

episode 6627.000000, reward total was -2.000000. running mean: 0.825944
episode 6628.000000, reward total was 3.000000. running mean: 0.847685
episode 6629.000000, reward total was 8.000000. running mean: 0.919208
episode 6630.000000, reward total was 2.000000. running mean: 0.930016
episode 6631.000000, reward total was 10.000000. running mean: 1.020716
episode 6632.000000, reward total was 4.000000. running mean: 1.050509
episode 6633.000000, reward total was 11.000000. running mean: 1.150004
episode 6634.000000, reward total was -17.000000. running mean: 0.968504
episode 6635.000000, reward total was 7.000000. running mean: 1.028819
episode 6636.000000, reward total was -3.000000. running mean: 0.988530
episode 6637.000000, reward total was -1.000000. running mean: 0.968645
episode 6638.000000, reward total was 5.000000. running mean: 1.008959
episode 6639.000000, reward total was 1.000000. running mean: 1.008869
episode 6640.000000, reward total was -1.000000. running mean: 0.98878

episode 6742.000000, reward total was 13.000000. running mean: 0.815233
episode 6743.000000, reward total was 9.000000. running mean: 0.897081
episode 6744.000000, reward total was 2.000000. running mean: 0.908110
episode 6745.000000, reward total was 8.000000. running mean: 0.979029
episode 6746.000000, reward total was 3.000000. running mean: 0.999239
episode 6747.000000, reward total was 5.000000. running mean: 1.039246
episode 6748.000000, reward total was 3.000000. running mean: 1.058854
episode 6749.000000, reward total was 2.000000. running mean: 1.068265
episode 6750.000000, reward total was 9.000000. running mean: 1.147583
episode 6751.000000, reward total was -4.000000. running mean: 1.096107
episode 6752.000000, reward total was -9.000000. running mean: 0.995146
episode 6753.000000, reward total was 11.000000. running mean: 1.095194
episode 6754.000000, reward total was 5.000000. running mean: 1.134242
episode 6755.000000, reward total was -3.000000. running mean: 1.092900
e

episode 6857.000000, reward total was -5.000000. running mean: 0.011372
episode 6858.000000, reward total was 4.000000. running mean: 0.051259
episode 6859.000000, reward total was 11.000000. running mean: 0.160746
episode 6860.000000, reward total was -10.000000. running mean: 0.059139
episode 6861.000000, reward total was -13.000000. running mean: -0.071453
episode 6862.000000, reward total was -2.000000. running mean: -0.090738
episode 6863.000000, reward total was -7.000000. running mean: -0.159831
episode 6864.000000, reward total was 8.000000. running mean: -0.078232
episode 6865.000000, reward total was 5.000000. running mean: -0.027450
episode 6866.000000, reward total was 4.000000. running mean: 0.012824
episode 6867.000000, reward total was 3.000000. running mean: 0.042696
episode 6868.000000, reward total was 2.000000. running mean: 0.062269
episode 6869.000000, reward total was 7.000000. running mean: 0.131646
episode 6870.000000, reward total was -11.000000. running mean: 

episode 6972.000000, reward total was 9.000000. running mean: 1.385064
episode 6973.000000, reward total was -1.000000. running mean: 1.361214
episode 6974.000000, reward total was 3.000000. running mean: 1.377601
episode 6975.000000, reward total was 6.000000. running mean: 1.423825
episode 6976.000000, reward total was -1.000000. running mean: 1.399587
episode 6977.000000, reward total was 11.000000. running mean: 1.495591
episode 6978.000000, reward total was 9.000000. running mean: 1.570635
episode 6979.000000, reward total was -7.000000. running mean: 1.484929
episode 6980.000000, reward total was -9.000000. running mean: 1.380080
episode 6981.000000, reward total was 7.000000. running mean: 1.436279
episode 6982.000000, reward total was 12.000000. running mean: 1.541916
episode 6983.000000, reward total was 4.000000. running mean: 1.566497
episode 6984.000000, reward total was 3.000000. running mean: 1.580832
episode 6985.000000, reward total was -5.000000. running mean: 1.515024